BERT: encoding, predicts words based on surrounding context.

GPT: decoding, uses context from left to right to predict the next word in a sentence

#**Low-resource Machine Translation using mBART50**

In [1]:
!pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu
# sentencepiece to encode,

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incomp

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
import evaluate
from transformers import (
    MBart50TokenizerFast,  # Tokenizer cho mô hình mBART50
    AutoModelForSeq2SeqLM,  # Mô hình Seq2Seq từ Hugging Face
    DataCollatorForSeq2Seq,  # Collator cho dữ liệu Seq2Seq
    Seq2SeqTrainingArguments,  # Các tham số huấn luyện cho mô hình Seq2Seq
    Seq2SeqTrainer  # Huấn luyện viên cho mô hình Seq2Seq
)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Dataset

In [4]:
# Tải bộ dữ liệu dịch thuật
# load from https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en
# data = load_dataset(
#     "mt_eng_vietnamese",  # Tên bộ dữ liệu
#     "iwslt2015-en-vi"  # Phiên bản của bộ dữ liệu
# )


In [5]:
!mkdir -p /content/drive/MyDrive/colab/data
%cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


In [6]:
# 1. Bước 1: Cài đặt và thiết lập Kaggle API
!pip install kaggle

# Tải file kaggle.json lên và đặt nó vào thư mục ~/.kaggle/
!mkdir -p ~/.kaggle
!mv /content/drive/MyDrive/colab/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


# !kaggle datasets download -d dhungvu/vinphomt


mv: cannot stat '/content/drive/MyDrive/colab/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [7]:
# Kiểm tra sự tồn tại của tệp `kaggle.json`
!ls ~/.kaggle/

# Kiểm tra nội dung của tệp `kaggle.json`
!cat ~/.kaggle/kaggle.json

# Kiểm tra quyền truy cập của tệp `kaggle.json`
!ls -l ~/.kaggle/

# # Bước 2: Sử dụng Kaggle API để tải dữ liệu
!kaggle datasets download -d cngonngc/phomt-datasetvinai


cat: /root/.kaggle/kaggle.json: No such file or directory
total 0
Dataset URL: https://www.kaggle.com/datasets/cngonngc/phomt-datasetvinai
License(s): Apache 2.0
phomt-datasetvinai.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
# Bước 3: Giải nén
import zipfile

# Giải nén tệp dữ liệu đã tải về
with zipfile.ZipFile('phomt-datasetvinai.zip', 'r') as zip_ref:
    zip_ref.extractall('phomt-datasetvinai')

# Kiểm tra các tệp đã được giải nén
!ls phomt-datasetvinai

dev.en	dev.vi	test.en  test.vi  train.en  train.vi


In [9]:
# TẠO DATASETDICT

import os
from datasets import Dataset, DatasetDict

# Định nghĩa hàm để tải dữ liệu từ các tập tin cục bộ
def load_phomt_dataset(data_dir):
    # Định nghĩa đường dẫn tới các tập tin dữ liệu
    train_file_en = os.path.join(data_dir, "train.en")
    train_file_vi = os.path.join(data_dir, "train.vi")
    val_file_en = os.path.join(data_dir, "dev.en")
    val_file_vi = os.path.join(data_dir, "dev.vi")
    test_file_en = os.path.join(data_dir, "test.en")
    test_file_vi = os.path.join(data_dir, "test.vi")

    # Đọc dữ liệu từ các tập tin
    def read_data(file_path_en, file_path_vi):
        src_texts = []
        tgt_texts = []
        with open(file_path_en, 'r', encoding='utf-8') as f_en, open(file_path_vi, 'r', encoding='utf-8') as f_vi:
            for line_en, line_vi in zip(f_en, f_vi):
                src_texts.append(line_en.strip())
                tgt_texts.append(line_vi.strip())
        return src_texts, tgt_texts

    train_src, train_tgt = read_data(train_file_en, train_file_vi)
    val_src, val_tgt = read_data(val_file_en, val_file_vi)
    test_src, test_tgt = read_data(test_file_en, test_file_vi)

    # Tạo dataset từ dữ liệu đã đọc
    dataset = DatasetDict({
        'train': Dataset.from_dict({'translation': [{'en': en, 'vi': vi} for en, vi in zip(train_src, train_tgt)]}),
        'validation': Dataset.from_dict({'translation': [{'en': en, 'vi': vi} for en, vi in zip(val_src, val_tgt)]}),
        'test': Dataset.from_dict({'translation': [{'en': en, 'vi': vi} for en, vi in zip(test_src, test_tgt)]})
    })

    return dataset

# Đường dẫn tới thư mục chứa các tập tin dữ liệu
data_dir = "phomt-datasetvinai"  # Thay thế bằng đường dẫn thực tế

# Tải dữ liệu PhoMT từ các tập tin cục bộ
phomt_data = load_phomt_dataset(data_dir)

# Hiển thị thông tin về bộ dữ liệu
print(phomt_data)


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 2977999
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 18719
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 19151
    })
})


In [10]:
# Lấy 1/22 dataset

from datasets import load_dataset, DatasetDict

# Function to sample a fraction of the dataset
def sample_dataset(dataset, fraction=1/22):
    return dataset.train_test_split(test_size=(1 - fraction))['train']

# Sample train, validation, and test sets
sampled_train = sample_dataset(phomt_data['train'])
sampled_validation = sample_dataset(phomt_data['validation'])
sampled_test = sample_dataset(phomt_data['test'])

# Combine the sampled datasets into a new DatasetDict
sampled_dataset = DatasetDict({
    'train': sampled_train,
    'validation': sampled_validation,
    'test': sampled_test
})

# Display the sampled dataset information
print(sampled_dataset)


# bộ cũ `iwslt2015-en-vi` chỉ có 133318-1269-1269 - CHỈ BẰNG 1/22 bộ cũ

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 135363
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 850
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 870
    })
})


In [11]:
data = sampled_dataset
print(data)
data['train'][0]  # Hiển thị mẫu đầu tiên của tập huấn luyện


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 135363
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 850
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 870
    })
})


{'translation': {'en': 'Notice if your skin feels hot.',
  'vi': 'Lưu ý dấu hiệu da nóng.'}}

```python
    def __getitem__(self, idx):
        return {
            "input_ids": self.src_input_ids[idx],   # Should put to this 2 parameters
            "labels": self.labels[idx]              # attention_mask
        }
```
Khi nào thì cần đẩy attention_mask vào?
- Question, Context => Thủ đô của Việt Nam là gì || Thủ đô Việt Nam là Hà Nội. (MRC: index, id, vị trí chứa đáp án phải cộng thêm bởi token của câu hỏi).
- Context => Question: (BERT 512 từ, mà ngữ cảnh thường dài => cắt bớt đi => encoding context riêng, encoding question riêng, => có 2 attention_mask, để nối => Cần tạo 1 attention_mask mới, bắt buộc phải đẩy attention_mask vào input_ids, ...).

##**Config**

In [12]:
## Config
# Định nghĩa lớp cấu hình cơ bản
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

# Định nghĩa lớp cấu hình cho bài toán dịch thuật
class NMTConfig(BaseConfig):
    # Data
    src_lang = 'en'  # Ngôn ngữ nguồn
    tgt_lang = 'vi'  # Ngôn ngữ đích
    max_len = 75  # Độ dài tối đa của câu
    add_special_tokens = True  # Thêm các token đặc biệt

    # Model
    model_name = "facebook/mbart-large-50-many-to-many-mmt"  # Tên mô hình

    # Training
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # Thiết bị (GPU hoặc CPU)
    learning_rate = 5e-5  # Tốc độ học
    train_batch_size = 16  # Kích thước batch khi huấn luyện
    eval_batch_size = 16  # Kích thước batch khi đánh giá
    num_train_epochs = 2  # Số epoch huấn luyện
    save_total_limit = 1  # Số lượng checkpoint lưu trữ
    ckpt_dir = f'./mbart50-{src_lang}-{tgt_lang}'  # Đường dẫn lưu trữ checkpoint
    eval_steps = 1000  # Số bước mỗi lần đánh giá

    # Inference
    beam_size = 5  # Kích thước beam search

cfg = NMTConfig()

##**Tokenizer, Model, Metric**

In [13]:
## test Dataset
# Định nghĩa tên mô hình và tải tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name, src_lang="en_XX", tgt_lang="vi_VN")
# Code for source_lang and target_lang.
# Example: sentence: "tôi đi học"
# {'input_ids': [250004, 2259, 2467, 2546, 2], 'attention_mask': [1, 1, 1, 1, 1]} # en_XX
# {'input_ids': [250024, 2259, 2467, 2546, 2], 'attention_mask': [1, 1, 1, 1, 1]} # vn_XX

sentence = "tôi đi học"
tokenizer(sentence)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

{'input_ids': [250004, 2259, 2467, 2546, 2], 'attention_mask': [1, 1, 1, 1, 1]}

In [14]:
# tokenizer = MBart50TokenizerFast.from_pretrained(cfg.model_name, src_lang="en_XX",tgt_lang = "vi_VN")
cfg.tokenizer = MBart50TokenizerFast.from_pretrained(cfg.model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_name)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [15]:
# Tải thư viện để đánh giá BLEU
metric = evaluate.load("sacrebleu")

# Hàm xử lý kết quả dự đoán và nhãn
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]  # Loại bỏ khoảng trắng thừa từ đầu và cuối của các câu dự đoán
    labels = [[label.strip()] for label in labels]  # Loại bỏ khoảng trắng thừa từ đầu và cuối của các câu nhãn và đóng gói lại thành danh sách

    return preds, labels  # Trả về danh sách các câu dự đoán và nhãn đã được xử lý

# Hàm tính toán các chỉ số đánh giá
def compute_metrics(eval_preds):
    preds, labels = eval_preds  # Tách các câu dự đoán và nhãn từ đầu vào
    if isinstance(preds, tuple):  # Kiểm tra xem câu dự đoán có phải là một tuple không
        preds = preds[0]  # Nếu phải, lấy phần tử đầu tiên

    preds = np.where(preds != -100, preds, cfg.tokenizer.pad_token_id)  # Thay thế các giá trị -100 bằng token điền thêm
    decoded_preds = cfg.tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)  # Giải mã các câu dự đoán và bỏ qua các token đặc biệt

    labels = np.where(labels != -100, labels, cfg.tokenizer.pad_token_id)  # Thay thế các giá trị -100 bằng token điền thêm
    decoded_labels = cfg.tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)  # Giải mã các câu nhãn và bỏ qua các token đặc biệt

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)  # Xử lý kết quả dự đoán và nhãn để loại bỏ khoảng trắng thừa

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)  # Tính điểm BLEU cho các câu dự đoán và nhãn
    result = {"bleu": result["score"]}  # Lưu điểm BLEU vào từ điển kết quả

    prediction_lens = [np.count_nonzero(pred != cfg.tokenizer.pad_token_id) for pred in preds]  # Tính độ dài của các câu dự đoán, bỏ qua các token điền thêm
    result["gen_len"] = np.mean(prediction_lens)  # Tính độ dài trung bình của các câu dự đoán và lưu vào từ điển kết quả
    result = {k: round(v, 4) for k, v in result.items()}  # Làm tròn các giá trị trong từ điển kết quả

    return result  # Trả về từ điển kết quả


##**Training**

Đoạn mã này định nghĩa một lớp `NMTDataset` (Neural Machine Translation Dataset) dùng để chuẩn bị dữ liệu cho mô hình dịch máy neural (Neural Machine Translation). Cụ thể, nó thực hiện các nhiệm vụ sau:

1. **Khởi tạo (`__init__`)**:
   - Nhận tham số cấu hình (`cfg`) và loại dữ liệu (`data_type`, mặc định là "train").
   - Đọc dữ liệu nguồn và đích từ tập dữ liệu (tập huấn luyện, tập kiểm thử, hoặc tập đánh giá) bằng phương thức `read_data`.
   - Chuyển đổi các đoạn văn bản nguồn và đích thành các chuỗi các ID đầu vào và nhãn bằng phương thức `texts_to_sequences`.

2. **Đọc dữ liệu (`read_data`)**:
   - Sử dụng thư viện `datasets` để tải tập dữ liệu dịch máy Anh - Việt (IWSLT2015) với `data_type` chỉ định.
   - Trích xuất các đoạn văn bản nguồn (`src_texts`) và văn bản đích (`tgt_texts`) từ tập dữ liệu dựa trên ngôn ngữ nguồn và ngôn ngữ đích được cấu hình (`cfg.src_lang` và `cfg.tgt_lang`).

3. **Chuyển đổi văn bản thành chuỗi (`texts_to_sequences`)**:
   - Sử dụng `tokenizer` từ cấu hình (`cfg.tokenizer`) để mã hóa các đoạn văn bản thành các ID đầu vào (`input_ids`).
   - Thực hiện padding và truncation để đảm bảo tất cả các chuỗi đều có độ dài cố định (`max_length`).

4. **Phương thức `__getitem__`**:
   - Trả về một dictionary chứa các ID đầu vào (`input_ids`) và nhãn (`labels`) cho một mẫu dữ liệu chỉ định bởi `idx`.

5. **Phương thức `__len__`**:
   - Trả về số lượng mẫu dữ liệu trong tập dữ liệu.

### Tại sao lại sử dụng lớp `NMTDataset` thay vì `DataDict`?

1. **Tính tiện dụng và cấu trúc**:
   - Lớp `NMTDataset` cung cấp một cấu trúc rõ ràng và có tổ chức cho việc quản lý và tiền xử lý dữ liệu.
   - Việc đọc dữ liệu, mã hóa văn bản, và truy xuất mẫu dữ liệu đều được đóng gói gọn gàng trong một lớp, giúp mã nguồn dễ đọc và bảo trì hơn.

2. **Khả năng mở rộng**:
   - Với lớp `NMTDataset`, bạn có thể dễ dàng mở rộng hoặc điều chỉnh phương thức đọc dữ liệu, tiền xử lý, hoặc cách thức mã hóa văn bản mà không ảnh hưởng đến các phần khác của mã nguồn.

3. **Tương thích với PyTorch**:
   - Lớp `NMTDataset` được kế thừa từ `Dataset` của PyTorch, cho phép sử dụng trực tiếp với `DataLoader` của PyTorch để tạo các batch dữ liệu và tối ưu hóa quá trình huấn luyện.

### Ví dụ sử dụng `NMTDataset`:

```python
from torch.utils.data import DataLoader

# Cấu hình ví dụ
class Config:
    src_lang = "en"
    tgt_lang = "vi"
    tokenizer = your_tokenizer  # Thay bằng tokenizer của bạn
    max_len = 128

cfg = Config()
dataset = NMTDataset(cfg, data_type="train")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    # Tiến hành huấn luyện với batch dữ liệu này
```

Sử dụng lớp `NMTDataset` giúp cho việc chuẩn bị dữ liệu cho mô hình dịch máy trở nên dễ dàng và hiệu quả hơn.

In [16]:
# colab để CPU chỉ được 1/22 dataset
# Kaggle chỉ để CPU: hết 29gb RAM CPU - THỬ ĐẨY QUA XÀI GPU: 73GB

# Kiểm tra nếu có GPU và thiết lập device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [17]:

# class NMTDataset(Dataset):
#     def __init__(self, cfg, data_type="train"):
#         super().__init__()
#         self.cfg = cfg

#         self.src_texts, self.tgt_texts = self.read_data(data_type)  # Đọc dữ liệu nguồn và đích

#         self.src_input_ids = self.texts_to_sequences(self.src_texts)  # Mã hóa các câu nguồn
#         self.labels = self.texts_to_sequences(self.tgt_texts)  # Mã hóa các câu đích

#     def read_data(self, data_type):
#         # data = load_dataset(
#         #     "mt_eng_vietnamese",
#         #     "iwslt2015-en-vi",   # 135k cặp câu
#         #     split=data_type
#         # )
#         src_texts = [sample["translation"][self.cfg.src_lang] for sample in data] # load list of source_sentences
#         tgt_texts = [sample["translation"][self.cfg.tgt_lang] for sample in data] # load list of target_sentences
#         return src_texts, tgt_texts

#     def texts_to_sequences(self, texts):
#         data_inputs = self.cfg.tokenizer(
#             texts,
#             padding='max_length',  # Điền thêm cho đủ độ dài tối đa
#             truncation=True,  # Cắt bớt nếu quá dài
#             max_length=self.cfg.max_len,  # Độ dài tối đa là 75
#             return_tensors='pt'  # Trả về tensor
#         )
#         return data_inputs.input_ids

#     def __getitem__(self, idx):
#         return {
#             "input_ids": self.src_input_ids[idx],  # Trả về các input_ids của câu nguồn
#             "labels": self.labels[idx]  # Trả về các nhãn của câu đích
#         }

#     def __len__(self):
#         return np.shape(self.src_input_ids)[0]  # Trả về số lượng mẫu

# -----------------------------------
# Thay vì sử dụng DATATYPE để load data từ bộ 2015 => ta sử dụng data load sample data từ 1/22 data gốc

from torch.utils.data import Dataset  # Lỗi TypeError: Dataset.__init__() missing 1 required positional argument: 'arrow_table' xảy ra
                                      # vì Dataset từ thư viện datasets của Hugging Face yêu cầu một đối số arrow_table khi khởi tạo, nhưng lớp NMTDataset không cung cấp đối số này trong phương thức __init__.
                                      # Cách khắc phục lỗi này bằng cách kế thừa từ torch.utils.data.Dataset thay vì datasets.Dataset
                                      # from torch.utils.data import Dataset

class NMTDataset(Dataset):
    def __init__(self, cfg, data):
        super().__init__()
        self.cfg = cfg
        self.src_texts, self.tgt_texts = self.read_data(data)  # Đọc dữ liệu nguồn và đích
        self.src_input_ids = self.texts_to_sequences(self.src_texts)  # Mã hóa các câu nguồn
        self.labels = self.texts_to_sequences(self.tgt_texts)  # Mã hóa các câu đích

    def read_data(self, data):
        src_texts = [sample["translation"][self.cfg.src_lang] for sample in data]  # Load list of source_sentences
        tgt_texts = [sample["translation"][self.cfg.tgt_lang] for sample in data]  # Load list of target_sentences
        return src_texts, tgt_texts

    def texts_to_sequences(self, texts):
        data_inputs = self.cfg.tokenizer(
            texts,
            padding='max_length',  # Điền thêm cho đủ độ dài tối đa
            truncation=True,  # Cắt bớt nếu quá dài
            max_length=self.cfg.max_len,  # Độ dài tối đa
            return_tensors='pt'  # Trả về tensor
        )
        return data_inputs.input_ids

    def __getitem__(self, idx):
        return {
            "input_ids": self.src_input_ids[idx],  # Trả về các input_ids của câu nguồn
            "labels": self.labels[idx]  # Trả về các nhãn của câu đích
        }

    def __len__(self):
        return len(self.src_input_ids)  # Trả về số lượng mẫu


print(data)
print(data['train'][0])  # Hiển thị mẫu đầu tiên của tập huấn luyện

# Chia nhỏ dữ liệu thành các tập huấn luyện, đánh giá và kiểm tra
train_data = data['train']
valid_data = data['validation']
test_data = data['test']

# Tạo dataset cho huấn luyện, đánh giá và kiểm tra
train_dataset = NMTDataset(cfg, train_data)
valid_dataset = NMTDataset(cfg, valid_data)
test_dataset = NMTDataset(cfg, test_data)

# Hiển thị thông tin về bộ dữ liệu
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Lỗi HỂT 12.7GB RAM khi load vào DATASET => thui, giảm đi 1/22 dataset



DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 135363
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 850
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 870
    })
})
{'translation': {'en': 'Notice if your skin feels hot.', 'vi': 'Lưu ý dấu hiệu da nóng.'}}
Train dataset size: 135363
Validation dataset size: 850
Test dataset size: 870


In [18]:
next(iter(train_dataset))

{'input_ids': tensor([250004,    438,  24494,   2174,    935,  21722,  12319,      7,   8010,
              5,      2,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1]),
 'labels': tensor([250004,  81737,   5604,  39973,   6842,     48,  50541,      5,      2,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,   

In [19]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    save_strategy='steps',
    save_steps=cfg.eval_steps,
    eval_steps=cfg.eval_steps,
    output_dir=cfg.ckpt_dir,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.eval_batch_size,
    learning_rate=cfg.learning_rate,
    save_total_limit=cfg.save_total_limit,
    num_train_epochs=cfg.num_train_epochs,
    load_best_model_at_end=True,
)

data_collator = DataCollatorForSeq2Seq(
    cfg.tokenizer,
    model=model
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=cfg.tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
from torch.utils.data import Dataset, DataLoader

# Sử dụng DataLoader để tải dữ liệu theo từng batch
train_loader = DataLoader(train_dataset, batch_size=cfg.train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=cfg.eval_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=cfg.eval_batch_size, shuffle=False)

# Kiểm tra kích thước của các tập dữ liệu
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Kiểm tra số batches in trainloder
print(f"Number of batches in train_loader: {len(train_loader)}")

# Kiểm tra một vài mẫu từ tập dữ liệu huấn luyện
for batch in train_loader:

    input_ids = batch['input_ids']
    labels = batch['labels']
    print("Input IDs:", input_ids)
    print("Labels:", labels)
    break  # Chỉ kiểm tra batch đầu tiên

# Kiểm tra cấu hình của mô hình và các thông số huấn luyện
print("---------------------------")
print("Model Configuration:")
print(f"Source Language: {cfg.src_lang}")
print(f"Target Language: {cfg.tgt_lang}")
print(f"Max Length: {cfg.max_len}")
print(f"Model Name: {cfg.model_name}")
print(f"Device: {cfg.device}")
print(f"Learning Rate: {cfg.learning_rate}")
print(f"Train Batch Size: {cfg.train_batch_size}")
print(f"Eval Batch Size: {cfg.eval_batch_size}")
print(f"Num Train Epochs: {cfg.num_train_epochs}")
print(f"Save Total Limit: {cfg.save_total_limit}")
print(f"Checkpoint Directory: {cfg.ckpt_dir}")
print(f"Eval Steps: {cfg.eval_steps}")
print(f"Beam Size: {cfg.beam_size}")

# Sau khi kiểm tra xong, bạn có thể tiến hành huấn luyện mô hình
# trainer.train()

Train dataset size: 135363
Validation dataset size: 850
Test dataset size: 870
Number of batches in train_loader: 8461
Input IDs: tensor([[250004,  43154,    136,  ...,      1,      1,      1],
        [250004,   5976,      5,  ...,      1,      1,      1],
        [250004,    384, 174404,  ...,      1,      1,      1],
        ...,
        [250004,   1061,    450,  ...,      1,      1,      1],
        [250004,  28541,  64113,  ...,      1,      1,      1],
        [250004,   3493,    450,  ...,      1,      1,      1]])
Labels: tensor([[250004,  39005,  19082,  ...,      1,      1,      1],
        [250004,  89106,    305,  ...,      1,      1,      1],
        [250004,  67935,    524,  ...,      1,      1,      1],
        ...,
        [250004,  62578,    580,  ...,      1,      1,      1],
        [250004,  81616,   3831,  ...,      1,      1,      1],
        [250004,  23598,   2275,  ...,      1,      1,      1]])
---------------------------
Model Configuration:
Source Language: 

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
prediction = trainer.predict(test_dataset)

In [ ]:
prediction

In [ ]:
def inference(
    text,
    tokenizer,
    model,
    device="cpu",
    max_length=75,
    beam_size=5
    ):
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
        )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        early_stopping=True,
        num_beams=beam_size,
        length_penalty=2.0
    )

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str

In [ ]:
sentence = 'i go to school'
inference(sentence, cfg.tokenizer, model)

##**Checkpoint**
https://drive.google.com/drive/folders/1ii_lPm2-1CfIhQM8RVzLgTHMxXDKgnk4?usp=sharing